# Applications of Machine Learning (2019 MOD006567 TRI2 F01CAM)

### Element 010

### Student ID: 0925739

## Component 2: Code for the Conversational Chatbot Project using LSTM networks

The mostly used libraries are first loaded in the cell below.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd

print( tf.__version__ )

2.2.0


The dataset for this project are in questions and answers format. Below loaded is the alexa topical chat dataset from where only first question and first answer from each conversation will be used to train the models in this project.

In [2]:
import json 
  
# Opening JSON file 
f = open('./alexa-prize-topical-chat-dataset-master/conversations/train.json') 
  
# returns JSON object as  
# a dictionary 
data = json.load(f) 
  

# Closing file 
f.close()

Below are some code written to extract the first question and answer from each conversation in the dataset.

In [3]:
convo = []
for key in data.keys():
    convo.append(data[key]['content'])

In [28]:
convo[0]

[{'message': 'Are you a fan of Google or Microsoft?',
  'agent': 'agent_1',
  'sentiment': 'Curious to dive deeper',
  'knowledge_source': ['FS1'],
  'turn_rating': 'Good'},
 {'message': 'Both are excellent technology they are helpful in many ways. For the security purpose both are super.',
  'agent': 'agent_2',
  'sentiment': 'Curious to dive deeper',
  'knowledge_source': ['FS1'],
  'turn_rating': 'Excellent'},
 {'message': "I'm not  a huge fan of Google, but I use it a lot because I have to. I think they are a monopoly in some sense. ",
  'agent': 'agent_1',
  'sentiment': 'Curious to dive deeper',
  'knowledge_source': ['FS1'],
  'turn_rating': 'Good'},
 {'message': 'Google provides online related services and products, which includes online ads, search engine and cloud computing.',
  'agent': 'agent_2',
  'sentiment': 'Curious to dive deeper',
  'knowledge_source': ['FS1'],
  'turn_rating': 'Excellent'},
 {'message': "Yeah, their services are good. I'm just not a fan of intrusive 

In [4]:
all_convo = []
for block in convo:
    one_convo = []
    for item in block:
        one_convo.append(item['message'])
    all_convo.append(one_convo)

Below is an example of a full conversation from the dataset.

In [31]:
all_convo[0]

['Are you a fan of Google or Microsoft?',
 'Both are excellent technology they are helpful in many ways. For the security purpose both are super.',
 "I'm not  a huge fan of Google, but I use it a lot because I have to. I think they are a monopoly in some sense. ",
 'Google provides online related services and products, which includes online ads, search engine and cloud computing.',
 "Yeah, their services are good. I'm just not a fan of intrusive they can be on our personal lives. ",
 'Google is leading the alphabet subsidiary and will continue to be the Umbrella company for Alphabet internet interest.',
 'Did you know Google had hundreds of live goats to cut the grass in the past? \n',
 'It is very interesting. Google provide "Chrome OS" which is a light weight OS. Google provided a lot of hardware mainly in 2010 to 2015. ',
 'I like Google Chrome. Do you use it as well for your browser? ',
 'Yes.Google is the biggest search engine and Google service figure out top 100 website, includi

Extracted below is the first question and answer from each covnersation and appended to questions and answers lists.

In [5]:
questions = []
for convo in all_convo:
    questions.append(convo[0])
        
answers = []
for convo in all_convo:
    answers.append(convo[1])

Below we import the chatterbot dataset which has about 839 questions and answers.

In [6]:
import os
import yaml

dir_path = 'raw_data'
files_list = os.listdir(dir_path + os.sep)

questions_chatterbot = list()
answers_chatterbot = list()
for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions_chatterbot.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers_chatterbot.append( ans )
        elif len( con )> 1:
            questions_chatterbot.append(con[0])
            answers_chatterbot.append(con[1])

The two datasets are combined below.

In [8]:
questions = questions_chatterbot+questions
answers = answers_chatterbot+answers

# answers_chatterbot[329] = "He's best known for the Dune novels, but he wrote more than that. Void is just as fascinating, and he was a prolific author of short stories too."
# answers_chatterbot[339] = "Fun fact: The Norby Chronicles was written primarily by his wife, Janet, but the publishers thought the only way they'd sell was if they added Isaac's name.  Kind of sad state of affairs, really."


Below, Keras tokenizer is used to tokenize the sentences returning sequences with indexes for each word. The indexes of the words are stored in a dictionary which can be mapped back. The length of the longest sentene is also calculated and the sequences less than max length is padded with zeros at the end to match the max length. encoder_input_data contains the padded question sequences.

In [32]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions ) 
tokenized_question_lines = tokenizer.texts_to_sequences( questions ) 
print('Sample tokenized: {}'.format(tokenized_question_lines[0]))


length_list = list()
for token_seq in tokenized_question_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
print( 'Questions max length is {} words'.format( max_input_length ))

padded_question_lines = preprocessing.sequence.pad_sequences( tokenized_question_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_question_lines )
print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))
print('Encoder input data sample-> {}'.format(encoder_input_data[0]))

question_word_dict = tokenizer.word_index
num_question_tokens = len( question_word_dict )+1
print( 'Number of Question tokens = {}'.format( num_question_tokens))
print('Dictionary: {}'.format(question_word_dict))

Sample tokenized: [26, 13, 933]
Questions max length is 66 words
Encoder input data shape -> (9194, 66)
Encoder input data sample-> [ 26  13 933   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
Number of Question tokens = 6424
Dictionary: {'you': 1, 'do': 2, 'the': 3, 'are': 4, 'a': 5, 'hi': 6, 'like': 7, 'to': 8, 'of': 9, 'how': 10, 'i': 11, 'hello': 12, 'is': 13, 'know': 14, 'there': 15, 'that': 16, 'in': 17, 'have': 18, 'about': 19, 'did': 20, 'it': 21, 'watch': 22, 'hey': 23, 'fan': 24, 'and': 25, 'what': 26, 'much': 27, 'was': 28, 'good': 29, 'think': 30, 'for': 31, 'on': 32, 'ever': 33, 'football': 34, 'use': 35, 'morning': 36, 'with': 37, 'basketball': 38, 'going': 39, 'today': 40, 'or': 41, 'music': 42, 'at': 43, 'enjoy': 44, 'doing': 45, 'so': 46, 'tv': 47, 'all': 48, 'an':

Printed above are the details of the processing that was carried out. At the end the dictionary created is also printed out.

Below processing is done for decoder input data.

In [33]:
answer_lines = []
for idx, line in enumerate(answers):
    print(idx,type(line))
    answer_lines.append( '<START> ' + line + ' <END>' )  

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( answer_lines ) 
tokenized_answer_lines = tokenizer.texts_to_sequences( answer_lines ) 

length_list = list()
for token_seq in tokenized_answer_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array( length_list ).max()
print( 'Answers max length is {}'.format( max_output_length ))

padded_answer_lines = preprocessing.sequence.pad_sequences( tokenized_answer_lines , maxlen=max_output_length, padding='post' )
decoder_input_data = np.array( padded_answer_lines )
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))
print(padded_answer_lines[:10])

answer_word_dict = tokenizer.word_index
num_answer_tokens = len( answer_word_dict )+1
print( 'Number of Answer tokens = {}'.format( num_answer_tokens))

print(list(answer_word_dict.values())[:10])

0 <class 'str'>
1 <class 'str'>
2 <class 'str'>
3 <class 'str'>
4 <class 'str'>
5 <class 'str'>
6 <class 'str'>
7 <class 'str'>
8 <class 'str'>
9 <class 'str'>
10 <class 'str'>
11 <class 'str'>
12 <class 'str'>
13 <class 'str'>
14 <class 'str'>
15 <class 'str'>
16 <class 'str'>
17 <class 'str'>
18 <class 'str'>
19 <class 'str'>
20 <class 'str'>
21 <class 'str'>
22 <class 'str'>
23 <class 'str'>
24 <class 'str'>
25 <class 'str'>
26 <class 'str'>
27 <class 'str'>
28 <class 'str'>
29 <class 'str'>
30 <class 'str'>
31 <class 'str'>
32 <class 'str'>
33 <class 'str'>
34 <class 'str'>
35 <class 'str'>
36 <class 'str'>
37 <class 'str'>
38 <class 'str'>
39 <class 'str'>
40 <class 'str'>
41 <class 'str'>
42 <class 'str'>
43 <class 'str'>
44 <class 'str'>
45 <class 'str'>
46 <class 'str'>
47 <class 'str'>
48 <class 'str'>
49 <class 'str'>
50 <class 'str'>
51 <class 'str'>
52 <class 'str'>
53 <class 'str'>
54 <class 'str'>
55 <class 'str'>
56 <class 'str'>
57 <class 'str'>
58 <class 'str'>
59 <cla

1988 <class 'str'>
1989 <class 'str'>
1990 <class 'str'>
1991 <class 'str'>
1992 <class 'str'>
1993 <class 'str'>
1994 <class 'str'>
1995 <class 'str'>
1996 <class 'str'>
1997 <class 'str'>
1998 <class 'str'>
1999 <class 'str'>
2000 <class 'str'>
2001 <class 'str'>
2002 <class 'str'>
2003 <class 'str'>
2004 <class 'str'>
2005 <class 'str'>
2006 <class 'str'>
2007 <class 'str'>
2008 <class 'str'>
2009 <class 'str'>
2010 <class 'str'>
2011 <class 'str'>
2012 <class 'str'>
2013 <class 'str'>
2014 <class 'str'>
2015 <class 'str'>
2016 <class 'str'>
2017 <class 'str'>
2018 <class 'str'>
2019 <class 'str'>
2020 <class 'str'>
2021 <class 'str'>
2022 <class 'str'>
2023 <class 'str'>
2024 <class 'str'>
2025 <class 'str'>
2026 <class 'str'>
2027 <class 'str'>
2028 <class 'str'>
2029 <class 'str'>
2030 <class 'str'>
2031 <class 'str'>
2032 <class 'str'>
2033 <class 'str'>
2034 <class 'str'>
2035 <class 'str'>
2036 <class 'str'>
2037 <class 'str'>
2038 <class 'str'>
2039 <class 'str'>
2040 <class 

3988 <class 'str'>
3989 <class 'str'>
3990 <class 'str'>
3991 <class 'str'>
3992 <class 'str'>
3993 <class 'str'>
3994 <class 'str'>
3995 <class 'str'>
3996 <class 'str'>
3997 <class 'str'>
3998 <class 'str'>
3999 <class 'str'>
4000 <class 'str'>
4001 <class 'str'>
4002 <class 'str'>
4003 <class 'str'>
4004 <class 'str'>
4005 <class 'str'>
4006 <class 'str'>
4007 <class 'str'>
4008 <class 'str'>
4009 <class 'str'>
4010 <class 'str'>
4011 <class 'str'>
4012 <class 'str'>
4013 <class 'str'>
4014 <class 'str'>
4015 <class 'str'>
4016 <class 'str'>
4017 <class 'str'>
4018 <class 'str'>
4019 <class 'str'>
4020 <class 'str'>
4021 <class 'str'>
4022 <class 'str'>
4023 <class 'str'>
4024 <class 'str'>
4025 <class 'str'>
4026 <class 'str'>
4027 <class 'str'>
4028 <class 'str'>
4029 <class 'str'>
4030 <class 'str'>
4031 <class 'str'>
4032 <class 'str'>
4033 <class 'str'>
4034 <class 'str'>
4035 <class 'str'>
4036 <class 'str'>
4037 <class 'str'>
4038 <class 'str'>
4039 <class 'str'>
4040 <class 

5988 <class 'str'>
5989 <class 'str'>
5990 <class 'str'>
5991 <class 'str'>
5992 <class 'str'>
5993 <class 'str'>
5994 <class 'str'>
5995 <class 'str'>
5996 <class 'str'>
5997 <class 'str'>
5998 <class 'str'>
5999 <class 'str'>
6000 <class 'str'>
6001 <class 'str'>
6002 <class 'str'>
6003 <class 'str'>
6004 <class 'str'>
6005 <class 'str'>
6006 <class 'str'>
6007 <class 'str'>
6008 <class 'str'>
6009 <class 'str'>
6010 <class 'str'>
6011 <class 'str'>
6012 <class 'str'>
6013 <class 'str'>
6014 <class 'str'>
6015 <class 'str'>
6016 <class 'str'>
6017 <class 'str'>
6018 <class 'str'>
6019 <class 'str'>
6020 <class 'str'>
6021 <class 'str'>
6022 <class 'str'>
6023 <class 'str'>
6024 <class 'str'>
6025 <class 'str'>
6026 <class 'str'>
6027 <class 'str'>
6028 <class 'str'>
6029 <class 'str'>
6030 <class 'str'>
6031 <class 'str'>
6032 <class 'str'>
6033 <class 'str'>
6034 <class 'str'>
6035 <class 'str'>
6036 <class 'str'>
6037 <class 'str'>
6038 <class 'str'>
6039 <class 'str'>
6040 <class 

7046 <class 'str'>
7047 <class 'str'>
7048 <class 'str'>
7049 <class 'str'>
7050 <class 'str'>
7051 <class 'str'>
7052 <class 'str'>
7053 <class 'str'>
7054 <class 'str'>
7055 <class 'str'>
7056 <class 'str'>
7057 <class 'str'>
7058 <class 'str'>
7059 <class 'str'>
7060 <class 'str'>
7061 <class 'str'>
7062 <class 'str'>
7063 <class 'str'>
7064 <class 'str'>
7065 <class 'str'>
7066 <class 'str'>
7067 <class 'str'>
7068 <class 'str'>
7069 <class 'str'>
7070 <class 'str'>
7071 <class 'str'>
7072 <class 'str'>
7073 <class 'str'>
7074 <class 'str'>
7075 <class 'str'>
7076 <class 'str'>
7077 <class 'str'>
7078 <class 'str'>
7079 <class 'str'>
7080 <class 'str'>
7081 <class 'str'>
7082 <class 'str'>
7083 <class 'str'>
7084 <class 'str'>
7085 <class 'str'>
7086 <class 'str'>
7087 <class 'str'>
7088 <class 'str'>
7089 <class 'str'>
7090 <class 'str'>
7091 <class 'str'>
7092 <class 'str'>
7093 <class 'str'>
7094 <class 'str'>
7095 <class 'str'>
7096 <class 'str'>
7097 <class 'str'>
7098 <class 

9123 <class 'str'>
9124 <class 'str'>
9125 <class 'str'>
9126 <class 'str'>
9127 <class 'str'>
9128 <class 'str'>
9129 <class 'str'>
9130 <class 'str'>
9131 <class 'str'>
9132 <class 'str'>
9133 <class 'str'>
9134 <class 'str'>
9135 <class 'str'>
9136 <class 'str'>
9137 <class 'str'>
9138 <class 'str'>
9139 <class 'str'>
9140 <class 'str'>
9141 <class 'str'>
9142 <class 'str'>
9143 <class 'str'>
9144 <class 'str'>
9145 <class 'str'>
9146 <class 'str'>
9147 <class 'str'>
9148 <class 'str'>
9149 <class 'str'>
9150 <class 'str'>
9151 <class 'str'>
9152 <class 'str'>
9153 <class 'str'>
9154 <class 'str'>
9155 <class 'str'>
9156 <class 'str'>
9157 <class 'str'>
9158 <class 'str'>
9159 <class 'str'>
9160 <class 'str'>
9161 <class 'str'>
9162 <class 'str'>
9163 <class 'str'>
9164 <class 'str'>
9165 <class 'str'>
9166 <class 'str'>
9167 <class 'str'>
9168 <class 'str'>
9169 <class 'str'>
9170 <class 'str'>
9171 <class 'str'>
9172 <class 'str'>
9173 <class 'str'>
9174 <class 'str'>
9175 <class 

Finally decoder target data is processed below without the <start> token for each tokenised answer sentence.

In [34]:
decoder_target_data = list()
for token_seq in tokenized_answer_lines:
    decoder_target_data.append(token_seq[ 1 : ]) 
    
padded_answer_lines = preprocessing.sequence.pad_sequences(decoder_target_data , maxlen=max_output_length, padding='post')
onehot_answer_lines = utils.to_categorical(padded_answer_lines, num_answer_tokens)
decoder_target_data = np.array(onehot_answer_lines)
print( 'Decoder target data shape -> {}'.format(decoder_target_data.shape))
print(padded_answer_lines[:10])
print(decoder_target_data[:10])

Decoder target data shape -> (9194, 86, 9033)
[[1284 2134   12    4 2549   11 2550   14  648 3214    8 4498 2551   10
    36    3    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [1438   12    4  759   11  648  150 1825  949   49  835 1285   14  451
    10 4499    4 2135   11    4  440  514    3    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [ 731   11    3    0    0    0    0    0    0    0    0    0   

Model is created below using the Keras functional API

In [35]:
import tensorflow as tf
tf.random.set_seed(4)

encoder_inputs = tf.keras.layers.Input(shape=( None, ))
encoder_embedding = tf.keras.layers.Embedding( num_question_tokens, 200, mask_zero=True) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200, return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None,  ))
decoder_embedding = tf.keras.layers.Embedding( num_answer_tokens, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200, return_state=True, return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding, initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_answer_tokens, activation=tf.keras.activations.softmax ) 
output = decoder_dense (decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy')

model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 200)    1284800     input_11[0][0]                   
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 200)    1806600     input_12[0][0]                   
____________________________________________________________________________________________

In [14]:
history = model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=250, epochs=10 )

Epoch 1/10
37/37 [==============================] - 241s 7s/step - loss: 0.3494
Epoch 2/10
37/37 [==============================] - 210s 6s/step - loss: 0.3384
Epoch 3/10
37/37 [==============================] - 244s 7s/step - loss: 0.3340
Epoch 4/10
37/37 [==============================] - 248s 7s/step - loss: 0.3297
Epoch 5/10
37/37 [==============================] - 246s 7s/step - loss: 0.3258
Epoch 6/10
37/37 [==============================] - 273s 7s/step - loss: 0.3214
Epoch 7/10
37/37 [==============================] - 304s 8s/step - loss: 0.3174
Epoch 8/10
37/37 [==============================] - 240s 6s/step - loss: 0.3137
Epoch 9/10
37/37 [==============================] - 216s 6s/step - loss: 0.3092
Epoch 10/10
37/37 [==============================] - 218s 6s/step - loss: 0.3063


Below defined is a fucntion that will make inference models based explicitly on the one trained above.

In [41]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

Similarly, another function below converts the use input sentences into tokens as per the dictionary created previously for input sequences and pads it upto the maximum inout sequence length so that the trained model can accept it.

In [42]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( question_word_dict[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')

Finally, a terminal chat input is created below untilt the user input is 'end'.

In [ ]:
enc_model , dec_model = make_inference_models()
user_input = ''
while not user_input == 'end':
    user_input = input( 'User Input : ' )
    states_values = enc_model.predict( str_to_tokens(user_input) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = answer_word_dict['start']
    stop_condition = False
    decoded_answer = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in answer_word_dict.items() :
            if sampled_word_index == index :
                decoded_answer += ' {}'.format( word )
                sampled_word = word

        if sampled_word == 'end' or len(decoded_answer.split()) > max_output_length:
            stop_condition = True

        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ]

    print('Bot Output : ', decoded_answer.strip('end') )